## MySQL - Airlines 2

Para el ejercicio de esta semana vamos a poblar las tablas de la base de datos generadas la semana pasada, usando la información de los siguientes archivos **.csv**:
- _**tabla_aerolinea.csv**_
- _**tabla_aeropuertos.csv**_
- _**tabla_distancias.csv**_
- _**tabla_estados.csv**_
- _**tabla_vuelos.csv**_

En **Python** escribe el código para poblar cada tabla. Toma en cuenta el tipo de dato para cada columna y si existen o no **constraints**.

Una vez llenas las tablas en **MySQL** contesta las siguiente preguntas usando código **SQL**:

1. ¿Que 5 aerolineas tienes el tamaño de flota más grande?
2. ¿Que distancia (en millas) tardaría en ir desde **ATL** a **JAN**?
3. ¿Cual es la distancia más grande entre 2 aeropuertos? 
4. ¿Qué ciudad tiene más aeropuertos?
5. Usando la tabla de **vuelos**, ¿Que **aerolinea** tuvo la mayor cantidad de vuelos en todas las fechas?
6. ¿Cuantas millas se recorrieron en total el día **2021-12-31**?
7. Muestra en orden los 5 días con más retrasos y la media de retraso para cada día.
8. Muestra los nombres de las 10 aerolineas que tenga menos retraso que la media.

**Escribe en el notebook el código de SQL para cada query.**

In [1]:
import numpy as np
import pandas as pd

import mysql.connector

df_aerolineas = pd.read_csv("tabla_aerolineas.csv")
df_aeropuertos = pd.read_csv("tabla_aeropuertos.csv")
df_distancias= pd.read_csv("tabla_distancias.csv")
df_estados = pd.read_csv("tabla_estados.csv")
df_vuelos= pd.read_csv("tabla_vuelos.csv")

Para realizar esta tarea primero debemos limpiar nuestros dataframes. Empezaremos cambiando los nombres de las columnas, para que coincidan con el nombre que tienen en la base de datos de SQL:

In [2]:
df_aerolineas = df_aerolineas.rename(columns={"ICAO": "icao", "IATA": "iata", "Group": "air_group", "Base": "base", "average_fleet_Age": "average_fleet_age"})

También debemos corregir el tipo de dato de las columnas founded, started_operations y fleet_size.

In [3]:
df_aerolineas['founded'] = df_aerolineas['founded'].astype('Int64')
df_aerolineas['started_operations'] = df_aerolineas['started_operations'].astype('Int64')
df_aerolineas['fleet_size'] = df_aerolineas['fleet_size'].astype('float64')

Ahora, en el dataframe de estados lo único que debemos corregir son las filas duplicadas, las cuales debemos eliminar. De esta manera, pasamos de tener 363 filas a sólo 54.

In [4]:
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   estado         363 non-null    object
 1   nombre_estado  363 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


In [5]:
df_estados["estado"].nunique()

54

In [6]:
df_estados = df_estados.drop_duplicates()
df_estados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, 0 to 339
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   estado         54 non-null     object
 1   nombre_estado  54 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


En el dataframe de aeropuertos volvemos a tener que cambiar el nombre de una columna:

In [7]:
df_aeropuertos = df_aeropuertos.rename(columns={"latitude": "latitud"})

Del dataframe de distancias eliminaremos las filas en los que hay valores faltantes, ya que no tiene sentido saber la distancia entre un aeropuerto y otro del que no sabemos su nombre.

In [8]:
df_distancias = df_distancias.dropna(subset=["aeropuerto_origen", "aeropuerto_destino"])

In [9]:
df_distancias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5934 entries, 0 to 5957
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   distancia_millas    5934 non-null   float64
 1   aeropuerto_origen   5934 non-null   object 
 2   aeropuerto_destino  5934 non-null   object 
dtypes: float64(1), object(2)
memory usage: 185.4+ KB


Además, eliminaremos las filas que contengan información duplicada:

In [10]:
df_distancias["sorted_key"] = df_distancias.apply(lambda row: tuple(sorted([row["aeropuerto_origen"], row["aeropuerto_destino"]])), axis=1)
df_distancias = df_distancias.drop_duplicates(subset=["sorted_key"]).drop(columns=["sorted_key"])

Ahora en el dataframe de vuelos debemos transformar los datos de la columna "fecha" a formato datetime.date y los datos de las columnas de horas a formato datetime.time:

In [11]:
df_vuelos["fecha"] = pd.to_datetime(df_vuelos["fecha"]).dt.date

columnas_time = ["hora_salida_programada", "hora_salida_real", "hora_despegue", "hora_llegada_real"]

for i in columnas_time:
    df_vuelos[i] = df_vuelos[i].replace("24:00", "00:00")
    df_vuelos[i] = pd.to_datetime(df_vuelos[i]).dt.time

C:\Users\Genesis\AppData\Local\Temp\ipykernel_5392\1364071989.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos[i] = pd.to_datetime(df_vuelos[i]).dt.time
C:\Users\Genesis\AppData\Local\Temp\ipykernel_5392\1364071989.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos[i] = pd.to_datetime(df_vuelos[i]).dt.time
C:\Users\Genesis\AppData\Local\Temp\ipykernel_5392\1364071989.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vuelos[i] = pd.to_datetime(df_vuelos[i]).dt.time
C:\Users\Genesis\AppData\Local\Temp\ipykernel_5392\1364071989.py:7: UserWarning: C

También eliminamos los valores duplicados de la columna número de vuelo, que es una PK en nuestra base de datos.

In [12]:
df_vuelos = df_vuelos.drop_duplicates(subset=['numero_vuelo'])

Por último, convertimos todos los valores NaN's en None:

In [13]:
dataframes = [df_aerolineas, df_aeropuertos, df_distancias, df_estados, df_vuelos]

for df in dataframes:
    df.fillna(np.nan, inplace=True)
    df.replace([np.nan], [None], inplace=True)

Hemos acabado de preparar los datos pero ahora nos queda llenar nuestra base de datos en SQL con los datos de nuestros dataframes. Para empleamos el siguiente script:

In [14]:
database = "airlines"

tablas_dfs = {"estados": df_estados, "aerolineas": df_aerolineas, "aeropuertos": df_aeropuertos, "distancias": df_distancias, "vuelos": df_vuelos}

batch_size = 1000  # Tuve que añadir un número máximo de filas por batch para no perder la conexión a MySQL

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = database)

cursor = db.cursor()

for tabla, df in tablas_dfs.items():
    cursor.execute(f"SELECT * FROM {tabla} LIMIT 0;")
    column_names = list(cursor.column_names)
    cursor.fetchall()

    insert_query = f"INSERT INTO {tabla} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")
    df = df.where(pd.notnull(df), None)
    values = [tuple(row) for row in df[column_names].values]

    for i in range(0, len(values), batch_size):
        batch = values[i : i + batch_size]
        cursor.executemany(insert_query, batch)
        db.commit()
        print(f"Añadidas: {cursor.rowcount} filas en '{tabla}' (Batch {i // batch_size + 1})")
        cursor.fetchall()

cursor.close()
db.close()

Añadidas: 54 filas en 'estados' (Batch 1)
Añadidas: 17 filas en 'aerolineas' (Batch 1)
Añadidas: 363 filas en 'aeropuertos' (Batch 1)
Añadidas: 1000 filas en 'distancias' (Batch 1)
Añadidas: 1000 filas en 'distancias' (Batch 2)
Añadidas: 977 filas en 'distancias' (Batch 3)
Añadidas: 1000 filas en 'vuelos' (Batch 1)
Añadidas: 1000 filas en 'vuelos' (Batch 2)
Añadidas: 1000 filas en 'vuelos' (Batch 3)
Añadidas: 1000 filas en 'vuelos' (Batch 4)
Añadidas: 1000 filas en 'vuelos' (Batch 5)
Añadidas: 1000 filas en 'vuelos' (Batch 6)
Añadidas: 394 filas en 'vuelos' (Batch 7)


Ahora que ya tenemos nuestra base de datos lista, procedemos a realizar las queries que nos piden y contestarlas:

1. ¿Que 5 aerolineas tienen el tamaño de flota más grande?

Delta Air Lines, American Airlines, United Airlines, Southwest Airlines y SkyWest Airlines.

2. ¿Que distancia (en millas) tardaría en ir desde **ATL** a **JAN**?

341 millas.

3. ¿Cual es la distancia más grande entre 2 aeropuertos? 

5095 millas.

4. ¿Qué ciudad tiene más aeropuertos?

Columbus.

5. Usando la tabla de **vuelos**, ¿Que **aerolinea** tuvo la mayor cantidad de vuelos en todas las fechas?

SWA.

6. ¿Cuantas millas se recorrieron en total el día **2021-12-31**?

6165 millas.

7. Muestra en orden los 5 días con más retrasos y la media de retraso para cada día.

2021-12-18, 2021-12-16, 2021-12-01, 2021-12-23 y 2021-12-05

8. Muestra los nombres de las 10 aerolineas que tenga menos retraso que la media.

Spirit Airlines, Republic Airways, Endeavor Air, Delta Air Lines, Envoy Air, Hawaiian Airlines, Frontier Airlines, SkyWest Airlines, Mesa Airlines y PSA Airlines.


In [ ]:
# Escribe aquí el código SQL para las queries.

-- ¿Que 5 aerolineas tienen el tamaño de flota más grande?
SELECT aerolinea, fleet_size
FROM aerolineas
ORDER BY fleet_size DESC
LIMIT 5;

-- ¿Que distancia (en millas) tardaría en ir desde **ATL** a **JAN**?
SELECT distancia_millas
FROM distancias
WHERE aeropuerto_origen LIKE 'ATL' AND aeropuerto_destino LIKE 'JAN';

-- ¿Cual es la distancia más grande entre 2 aeropuertos? 
SELECT distancia_millas
FROM distancias
ORDER BY distancia_millas DESC
LIMIT 1;

-- ¿Qué ciudad tiene más aeropuertos?
SELECT ciudad, COUNT(codigo_aeropuerto) AS n_aeropuertos
FROM aeropuertos
GROUP BY ciudad
ORDER BY n_aeropuertos DESC
LIMIT 1;

-- Usando la tabla de **vuelos**, ¿Que **aerolinea** tuvo la mayor cantidad de vuelos en todas las fechas?
SELECT aerolinea, COUNT(*) AS n_vuelos
FROM vuelos
GROUP BY aerolinea
ORDER BY n_vuelos DESC
LIMIT 1;

-- ¿Cuantas millas se recorrieron en total el día **2021-12-31**?
SELECT SUM(distancia_millas)
FROM vuelos v
INNER JOIN distancias d ON v.aeropuerto_origen = d.aeropuerto_origen AND v.aeropuerto_destino = d.aeropuerto_destino
WHERE fecha = "2021-12-31"
;

-- Muestra en orden los 5 días con más retrasos y la media de retraso para cada día.
SELECT fecha, SUM(retraso_llegada) AS retraso, AVG(retraso_llegada) AS media_retraso
FROM vuelos
GROUP BY fecha
ORDER BY retraso DESC
LIMIT 5;

-- Muestra los nombres de las 10 aerolineas que tenga menos retraso que la media.
SELECT a.aerolinea, AVG(retraso_llegada) AS media_retraso
FROM vuelos v
INNER JOIN aerolineas a ON v.aerolinea=a.icao
GROUP BY aerolinea
HAVING media_retraso < (
    SELECT AVG(retraso_llegada) FROM vuelos
)
ORDER BY media_retraso ASC
LIMIT 10;

In [17]:
##############################################################################################################################